In [1]:
from sklearn import datasets
from scipy.io.arff import loadarff
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd


/home/yannick/bin/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
forest = datasets.fetch_covtype()

In [3]:
type(forest)

sklearn.datasets.base.Bunch

In [4]:
X, X_test, y, y_test = train_test_split(
    forest.data, forest.target == 2, test_size=0.3, random_state=1234)

In [5]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

(406708, 54) (406708,) (174304, 54) (174304,)


## Maintenant un peu de xgboost!

### D'abord un tout petit

In [6]:
import numpy as np
def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

In [7]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 4
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815255	train-auc:0.816074
[3]	eval-auc:0.866843	train-auc:0.867408


In [8]:
X_test[0, [0,25,35,15]]

array([ 2687.,     0.,     0.,     0.])

In [9]:
leaves = bst.predict(dtest, pred_leaf=True)
z = bst.predict(dtest)
leaves[:3], z[:3], inv_logit(0.388039+ 0.354413)

(array([[73, 77, 78, 81],
        [73, 78, 76, 88],
        [73, 77, 78, 82]], dtype=int32),
 array([ 0.75847626,  0.63132304,  0.7231403 ], dtype=float32),
 0.67753180825889836)

In [10]:
leaves[400], y[400], X_test[400, [0,7,3]]

(array([ 97, 102,  99,  89], dtype=int32),
 False,
 array([ 3126.,   254.,   283.]))

In [11]:
bst.dump_model("bst_2_trees")
! cat bst_2_trees

booster[0]:
0:[f0<3074] yes=1,no=2,missing=1
	1:[f0<2501] yes=3,no=4,missing=3
		3:[f0<2466] yes=7,no=8,missing=7
			7:[f44<0.5] yes=15,no=16,missing=15
				15:[f23<0.5] yes=31,no=32,missing=31
					31:[f46<0.5] yes=61,no=62,missing=61
						61:leaf=-0.587316
						62:leaf=0.381818
					32:[f12<0.5] yes=63,no=64,missing=63
						63:leaf=-0.537327
						64:leaf=-0.257143
				16:leaf=0.547826
			8:[f10<0.5] yes=17,no=18,missing=17
				17:[f23<0.5] yes=33,no=34,missing=33
					33:[f46<0.5] yes=65,no=66,missing=65
						65:leaf=-0.479532
						66:leaf=0.447273
					34:[f5<1358] yes=67,no=68,missing=67
						67:leaf=-0.0828358
						68:leaf=-0.573723
				18:[f9<5146] yes=35,no=36,missing=35
					35:[f5<760] yes=69,no=70,missing=69
						69:leaf=0.218182
						70:leaf=0.58209
					36:[f1<155] yes=71,no=72,missing=71
						71:leaf=-0.546667
						72:leaf=-0.0666667
		4:[f25<0.5] yes=9,no=10,missing=9
			9:[f35<0.5] yes=19,no=20,missing=19
				19:[f15<0.5] yes=37,no=38,missing=37
					37:[f36<

In [12]:
X.shape, y.shape, X_test.shape, y_test.shape

((406708, 54), (406708,), (174304, 54), (174304,))

In [16]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815255	train-auc:0.816074
[40]	eval-auc:0.922714	train-auc:0.925346
[80]	eval-auc:0.944923	train-auc:0.948678
[120]	eval-auc:0.957583	train-auc:0.96204
[160]	eval-auc:0.964993	train-auc:0.969911
[200]	eval-auc:0.97121	train-auc:0.976483
[240]	eval-auc:0.975295	train-auc:0.980906
[280]	eval-auc:0.97817	train-auc:0.983926
[320]	eval-auc:0.980859	train-auc:0.986674
[360]	eval-auc:0.982833	train-auc:0.98864
[400]	eval-auc:0.98469	train-auc:0.990505
[440]	eval-auc:0.985995	train-auc:0.991836
[480]	eval-auc:0.987254	train-auc:0.993098
[520]	eval-auc:0.988187	train-auc:0.993982
[560]	eval-auc:0.989098	train-auc:0.994851
[600]	eval-auc:0.98963	train-auc:0.995409
CPU times: user 2min 51s, sys: 5.6 s, total: 2min 56s
Wall time: 44.4 s


In [17]:
%%time

import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'hist'
}

num_round = 601
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.815251	train-auc:0.816075
[40]	eval-auc:0.9265	train-auc:0.929326
[80]	eval-auc:0.94302	train-auc:0.946318
[120]	eval-auc:0.956107	train-auc:0.960191
[160]	eval-auc:0.964489	train-auc:0.969099
[200]	eval-auc:0.970554	train-auc:0.975652
[240]	eval-auc:0.975686	train-auc:0.98084
[280]	eval-auc:0.978581	train-auc:0.983946
[320]	eval-auc:0.981304	train-auc:0.98677
[360]	eval-auc:0.983529	train-auc:0.989086
[400]	eval-auc:0.984916	train-auc:0.990508
[440]	eval-auc:0.986242	train-auc:0.991837
[480]	eval-auc:0.987173	train-auc:0.992835
[520]	eval-auc:0.988132	train-auc:0.993809
[560]	eval-auc:0.988981	train-auc:0.99464
[600]	eval-auc:0.989828	train-auc:0.995467
CPU times: user 3min 23s, sys: 1.36 s, total: 3min 24s
Wall time: 51.3 s


In [21]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.830985	train-auc:0.830948
[40]	eval-auc:0.915693	train-auc:0.918135
[80]	eval-auc:0.939123	train-auc:0.94286
[120]	eval-auc:0.951646	train-auc:0.95623
[160]	eval-auc:0.960016	train-auc:0.965135
[200]	eval-auc:0.966127	train-auc:0.971741
[240]	eval-auc:0.970492	train-auc:0.976488
[280]	eval-auc:0.973321	train-auc:0.979597
[320]	eval-auc:0.975859	train-auc:0.982351
[360]	eval-auc:0.978057	train-auc:0.984724
[400]	eval-auc:0.980107	train-auc:0.986822
[440]	eval-auc:0.981788	train-auc:0.988533
[480]	eval-auc:0.982918	train-auc:0.98977
[520]	eval-auc:0.984031	train-auc:0.99096
[560]	eval-auc:0.985204	train-auc:0.992121
[600]	eval-auc:0.986085	train-auc:0.99296
[640]	eval-auc:0.986882	train-auc:0.993774
[680]	eval-auc:0.987448	train-auc:0.994378
[720]	eval-auc:0.987992	train-auc:0.994973
[760]	eval-auc:0.988323	train-auc:0.995418
[800]	eval-auc:0.988865	train-auc:0.995881
[840]	eval-auc:0.989195	train-auc:0.996257
[880]	eval-auc:0.98952	train-auc:0.996576
[920]	eval-auc:0.9898

In [22]:
%%time 

param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.15,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 1001
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.813996	train-auc:0.814956
[40]	eval-auc:0.916638	train-auc:0.919448
[80]	eval-auc:0.936904	train-auc:0.940796
[120]	eval-auc:0.949872	train-auc:0.954719
[160]	eval-auc:0.958275	train-auc:0.963817
[200]	eval-auc:0.964482	train-auc:0.970541
[240]	eval-auc:0.969145	train-auc:0.975591
[280]	eval-auc:0.972509	train-auc:0.97925
[320]	eval-auc:0.97558	train-auc:0.982673
[360]	eval-auc:0.978116	train-auc:0.98538
[400]	eval-auc:0.979837	train-auc:0.987093
[440]	eval-auc:0.981466	train-auc:0.988841
[480]	eval-auc:0.982602	train-auc:0.990082
[520]	eval-auc:0.98392	train-auc:0.991358
[560]	eval-auc:0.985042	train-auc:0.992526
[600]	eval-auc:0.985612	train-auc:0.993219
[640]	eval-auc:0.986175	train-auc:0.99391
[680]	eval-auc:0.986757	train-auc:0.994553
[720]	eval-auc:0.987361	train-auc:0.995142
[760]	eval-auc:0.987794	train-auc:0.995653
[800]	eval-auc:0.988232	train-auc:0.996087
[840]	eval-auc:0.988591	train-auc:0.996443
[880]	eval-auc:0.988937	train-auc:0.996772
[920]	eval-auc:0.989

In [26]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 40
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

Start training...
[1]	valid_0's auc: 0.837071	valid_0's l2: 0.2118
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.852732	valid_0's l2: 0.187965
[3]	valid_0's auc: 0.860523	valid_0's l2: 0.172299
[4]	valid_0's auc: 0.864331	valid_0's l2: 0.162905
[5]	valid_0's auc: 0.868307	valid_0's l2: 0.155853
[6]	valid_0's auc: 0.873552	valid_0's l2: 0.151438
[7]	valid_0's auc: 0.876429	valid_0's l2: 0.147185
[8]	valid_0's auc: 0.880987	valid_0's l2: 0.143569
[9]	valid_0's auc: 0.882774	valid_0's l2: 0.14131
[10]	valid_0's auc: 0.885566	valid_0's l2: 0.13925
[11]	valid_0's auc: 0.887663	valid_0's l2: 0.137781
[12]	valid_0's auc: 0.889575	valid_0's l2: 0.136255
[13]	valid_0's auc: 0.891339	valid_0's l2: 0.135098
[14]	valid_0's auc: 0.893987	valid_0's l2: 0.133428
[15]	valid_0's auc: 0.89599	valid_0's l2: 0.132204
[16]	valid_0's auc: 0.896944	valid_0's l2: 0.131429
[17]	valid_0's auc: 0.898214	valid_0's l2: 0.130408
[18]	valid_0's auc: 0.899322	valid_0's l2: 0.129624

[166]	valid_0's auc: 0.960393	valid_0's l2: 0.0833771
[167]	valid_0's auc: 0.960568	valid_0's l2: 0.0832319
[168]	valid_0's auc: 0.960808	valid_0's l2: 0.0829827
[169]	valid_0's auc: 0.960893	valid_0's l2: 0.0829036
[170]	valid_0's auc: 0.960979	valid_0's l2: 0.0828435
[171]	valid_0's auc: 0.961062	valid_0's l2: 0.0827583
[172]	valid_0's auc: 0.961178	valid_0's l2: 0.0826476
[173]	valid_0's auc: 0.961302	valid_0's l2: 0.0825533
[174]	valid_0's auc: 0.96132	valid_0's l2: 0.0825273
[175]	valid_0's auc: 0.961417	valid_0's l2: 0.0824255
[176]	valid_0's auc: 0.961523	valid_0's l2: 0.0823366
[177]	valid_0's auc: 0.961717	valid_0's l2: 0.0821382
[178]	valid_0's auc: 0.961751	valid_0's l2: 0.0820924
[179]	valid_0's auc: 0.961855	valid_0's l2: 0.0819759
[180]	valid_0's auc: 0.961946	valid_0's l2: 0.0818926
[181]	valid_0's auc: 0.962063	valid_0's l2: 0.0817514
[182]	valid_0's auc: 0.962131	valid_0's l2: 0.081687
[183]	valid_0's auc: 0.962168	valid_0's l2: 0.081647
[184]	valid_0's auc: 0.962191	v

[320]	valid_0's auc: 0.974801	valid_0's l2: 0.0677129
[321]	valid_0's auc: 0.974853	valid_0's l2: 0.0676483
[322]	valid_0's auc: 0.974861	valid_0's l2: 0.0676361
[323]	valid_0's auc: 0.974892	valid_0's l2: 0.0675901
[324]	valid_0's auc: 0.974978	valid_0's l2: 0.0674872
[325]	valid_0's auc: 0.975044	valid_0's l2: 0.0674176
[326]	valid_0's auc: 0.97515	valid_0's l2: 0.0672935
[327]	valid_0's auc: 0.975176	valid_0's l2: 0.0672493
[328]	valid_0's auc: 0.975213	valid_0's l2: 0.067188
[329]	valid_0's auc: 0.975269	valid_0's l2: 0.0671114
[330]	valid_0's auc: 0.975329	valid_0's l2: 0.0670322
[331]	valid_0's auc: 0.975386	valid_0's l2: 0.0669432
[332]	valid_0's auc: 0.975482	valid_0's l2: 0.0668476
[333]	valid_0's auc: 0.97553	valid_0's l2: 0.0667882
[334]	valid_0's auc: 0.97559	valid_0's l2: 0.0667322
[335]	valid_0's auc: 0.975649	valid_0's l2: 0.066651
[336]	valid_0's auc: 0.97577	valid_0's l2: 0.0665093
[337]	valid_0's auc: 0.975796	valid_0's l2: 0.0664756
[338]	valid_0's auc: 0.975845	vali

[474]	valid_0's auc: 0.981294	valid_0's l2: 0.0590727
[475]	valid_0's auc: 0.981381	valid_0's l2: 0.0589614
[476]	valid_0's auc: 0.981411	valid_0's l2: 0.0589212
[477]	valid_0's auc: 0.981443	valid_0's l2: 0.0588745
[478]	valid_0's auc: 0.981473	valid_0's l2: 0.0588423
[479]	valid_0's auc: 0.981498	valid_0's l2: 0.0588026
[480]	valid_0's auc: 0.981505	valid_0's l2: 0.0587978
[481]	valid_0's auc: 0.98154	valid_0's l2: 0.058756
[482]	valid_0's auc: 0.981589	valid_0's l2: 0.058707
[483]	valid_0's auc: 0.981592	valid_0's l2: 0.0587022
[484]	valid_0's auc: 0.981616	valid_0's l2: 0.058664
[485]	valid_0's auc: 0.981632	valid_0's l2: 0.0586245
[486]	valid_0's auc: 0.981659	valid_0's l2: 0.0585634
[487]	valid_0's auc: 0.981677	valid_0's l2: 0.0585356
[488]	valid_0's auc: 0.981721	valid_0's l2: 0.0584779
[489]	valid_0's auc: 0.981723	valid_0's l2: 0.0584711
[490]	valid_0's auc: 0.981791	valid_0's l2: 0.0583608
[491]	valid_0's auc: 0.981807	valid_0's l2: 0.0583379
[492]	valid_0's auc: 0.981816	va

[634]	valid_0's auc: 0.985276	valid_0's l2: 0.0530293
[635]	valid_0's auc: 0.985297	valid_0's l2: 0.0529818
[636]	valid_0's auc: 0.985329	valid_0's l2: 0.0529178
[637]	valid_0's auc: 0.985334	valid_0's l2: 0.0529082
[638]	valid_0's auc: 0.985338	valid_0's l2: 0.0528915
[639]	valid_0's auc: 0.985368	valid_0's l2: 0.0528325
[640]	valid_0's auc: 0.985391	valid_0's l2: 0.0528061
[641]	valid_0's auc: 0.985405	valid_0's l2: 0.0527802
[642]	valid_0's auc: 0.985423	valid_0's l2: 0.0527461
[643]	valid_0's auc: 0.985439	valid_0's l2: 0.0527235
[644]	valid_0's auc: 0.985441	valid_0's l2: 0.0527179
[645]	valid_0's auc: 0.985453	valid_0's l2: 0.0526905
[646]	valid_0's auc: 0.985482	valid_0's l2: 0.0526294
[647]	valid_0's auc: 0.985512	valid_0's l2: 0.052583
[648]	valid_0's auc: 0.985511	valid_0's l2: 0.0525613
[649]	valid_0's auc: 0.985553	valid_0's l2: 0.0525019
[650]	valid_0's auc: 0.985558	valid_0's l2: 0.0524796
[651]	valid_0's auc: 0.98557	valid_0's l2: 0.0524598
[652]	valid_0's auc: 0.985571	

...

[1104]	valid_0's auc: 0.990456	valid_0's l2: 0.0433903
[1105]	valid_0's auc: 0.990461	valid_0's l2: 0.0433761
[1106]	valid_0's auc: 0.990469	valid_0's l2: 0.0433641
[1107]	valid_0's auc: 0.990472	valid_0's l2: 0.0433474
[1108]	valid_0's auc: 0.990472	valid_0's l2: 0.0433415
[1109]	valid_0's auc: 0.990476	valid_0's l2: 0.0433337
[1110]	valid_0's auc: 0.990476	valid_0's l2: 0.0433351
[1111]	valid_0's auc: 0.990479	valid_0's l2: 0.0433269
[1112]	valid_0's auc: 0.990482	valid_0's l2: 0.0433201
[1113]	valid_0's auc: 0.990483	valid_0's l2: 0.0433107
[1114]	valid_0's auc: 0.990484	valid_0's l2: 0.0433079
[1115]	valid_0's auc: 0.990484	valid_0's l2: 0.0433076
[1116]	valid_0's auc: 0.990495	valid_0's l2: 0.0432879
[1117]	valid_0's auc: 0.990496	valid_0's l2: 0.0432846
[1118]	valid_0's auc: 0.990498	valid_0's l2: 0.0432805
[1119]	valid_0's auc: 0.990503	valid_0's l2: 0.0432679
[1120]	valid_0's auc: 0.99051	valid_0's l2: 0.0432532
[1121]	valid_0's auc: 0.990516	valid_0's l2: 0.0432415
[1122]	vali

[1263]	valid_0's auc: 0.991145	valid_0's l2: 0.041859
[1264]	valid_0's auc: 0.991145	valid_0's l2: 0.041851
[1265]	valid_0's auc: 0.991148	valid_0's l2: 0.0418459
[1266]	valid_0's auc: 0.991148	valid_0's l2: 0.0418454
[1267]	valid_0's auc: 0.991152	valid_0's l2: 0.0418368
[1268]	valid_0's auc: 0.991163	valid_0's l2: 0.0418178
[1269]	valid_0's auc: 0.991176	valid_0's l2: 0.0417982
[1270]	valid_0's auc: 0.991181	valid_0's l2: 0.0417857
[1271]	valid_0's auc: 0.991188	valid_0's l2: 0.0417654
[1272]	valid_0's auc: 0.991193	valid_0's l2: 0.0417563
[1273]	valid_0's auc: 0.9912	valid_0's l2: 0.0417423
[1274]	valid_0's auc: 0.991201	valid_0's l2: 0.0417408
[1275]	valid_0's auc: 0.991201	valid_0's l2: 0.0417394
[1276]	valid_0's auc: 0.991209	valid_0's l2: 0.0417318
[1277]	valid_0's auc: 0.99121	valid_0's l2: 0.0417301
[1278]	valid_0's auc: 0.991211	valid_0's l2: 0.0417263
[1279]	valid_0's auc: 0.991217	valid_0's l2: 0.0417202
[1280]	valid_0's auc: 0.991224	valid_0's l2: 0.0417071
[1281]	valid_0'

[1419]	valid_0's auc: 0.991933	valid_0's l2: 0.0401255
[1420]	valid_0's auc: 0.991936	valid_0's l2: 0.0401161
[1421]	valid_0's auc: 0.991941	valid_0's l2: 0.0401106
[1422]	valid_0's auc: 0.991944	valid_0's l2: 0.040101
[1423]	valid_0's auc: 0.991945	valid_0's l2: 0.0400973
[1424]	valid_0's auc: 0.991945	valid_0's l2: 0.0400975
[1425]	valid_0's auc: 0.991947	valid_0's l2: 0.0400933
[1426]	valid_0's auc: 0.991947	valid_0's l2: 0.0400925
[1427]	valid_0's auc: 0.991947	valid_0's l2: 0.040089
[1428]	valid_0's auc: 0.991948	valid_0's l2: 0.0400876
[1429]	valid_0's auc: 0.991954	valid_0's l2: 0.0400697
[1430]	valid_0's auc: 0.991955	valid_0's l2: 0.0400664
[1431]	valid_0's auc: 0.991956	valid_0's l2: 0.0400627
[1432]	valid_0's auc: 0.991957	valid_0's l2: 0.0400621
[1433]	valid_0's auc: 0.99196	valid_0's l2: 0.0400553
[1434]	valid_0's auc: 0.991963	valid_0's l2: 0.0400476
[1435]	valid_0's auc: 0.991969	valid_0's l2: 0.0400363
[1436]	valid_0's auc: 0.991971	valid_0's l2: 0.0400356
[1437]	valid_

[1572]	valid_0's auc: 0.992468	valid_0's l2: 0.038818
[1573]	valid_0's auc: 0.992471	valid_0's l2: 0.0388149
[1574]	valid_0's auc: 0.992478	valid_0's l2: 0.0387978
[1575]	valid_0's auc: 0.99248	valid_0's l2: 0.0387946
[1576]	valid_0's auc: 0.992481	valid_0's l2: 0.0387925
[1577]	valid_0's auc: 0.992481	valid_0's l2: 0.0387914
[1578]	valid_0's auc: 0.992484	valid_0's l2: 0.0387882
[1579]	valid_0's auc: 0.992484	valid_0's l2: 0.0387861
[1580]	valid_0's auc: 0.992486	valid_0's l2: 0.0387808
[1581]	valid_0's auc: 0.992488	valid_0's l2: 0.0387738
[1582]	valid_0's auc: 0.99249	valid_0's l2: 0.0387678
[1583]	valid_0's auc: 0.992493	valid_0's l2: 0.038762
[1584]	valid_0's auc: 0.992492	valid_0's l2: 0.0387617
[1585]	valid_0's auc: 0.992492	valid_0's l2: 0.0387593
[1586]	valid_0's auc: 0.992493	valid_0's l2: 0.0387579
[1587]	valid_0's auc: 0.992497	valid_0's l2: 0.0387502
[1588]	valid_0's auc: 0.992512	valid_0's l2: 0.0387289
[1589]	valid_0's auc: 0.992542	valid_0's l2: 0.0386836
[1590]	valid_0

[1729]	valid_0's auc: 0.992919	valid_0's l2: 0.0377153
[1730]	valid_0's auc: 0.992921	valid_0's l2: 0.0377089
[1731]	valid_0's auc: 0.992923	valid_0's l2: 0.0377004
[1732]	valid_0's auc: 0.992923	valid_0's l2: 0.0376899
[1733]	valid_0's auc: 0.992924	valid_0's l2: 0.0376865
[1734]	valid_0's auc: 0.992933	valid_0's l2: 0.0376606
[1735]	valid_0's auc: 0.992935	valid_0's l2: 0.0376559
[1736]	valid_0's auc: 0.992937	valid_0's l2: 0.0376435
[1737]	valid_0's auc: 0.992941	valid_0's l2: 0.0376272
[1738]	valid_0's auc: 0.992944	valid_0's l2: 0.0376239
[1739]	valid_0's auc: 0.992946	valid_0's l2: 0.0376227
[1740]	valid_0's auc: 0.992951	valid_0's l2: 0.0376118
[1741]	valid_0's auc: 0.992955	valid_0's l2: 0.0376043
[1742]	valid_0's auc: 0.992971	valid_0's l2: 0.0375895
[1743]	valid_0's auc: 0.99297	valid_0's l2: 0.037591
[1744]	valid_0's auc: 0.992973	valid_0's l2: 0.0375834
[1745]	valid_0's auc: 0.992985	valid_0's l2: 0.0375612
[1746]	valid_0's auc: 0.992987	valid_0's l2: 0.0375581
[1747]	valid

In [36]:
%%time

import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

# create dataset for lightgbm
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.25,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'device': 'gpu'
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5,
                verbose_eval=200)

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

Start training...


LightGBMError: b'GPU Tree Learner was not enabled in this build. Recompile with CMake option -DUSE_GPU=1'